In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [90]:
df = pd.read_csv('final_bus_routes.csv')
df['group'] = df['bus'].apply(lambda x: x[2:8]) #remove bus number

unique_routes = df['bus'].unique()
cmap = plt.get_cmap('viridis')
colors = [cmap(i / len(unique_routes)) for i in range(len(unique_routes))]
color_map = {route: f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}' for route, (r, g, b, _) in zip(unique_routes, colors)}

def plot_bus_route(filtered_df, map_object):
    coordinates = []
    for index, row in filtered_df.iterrows():
        coords = [row['lat'], row['lon']]
        coordinates.append(coords)
        route_color = color_map[row['bus']]
        tooltip_content = f"<b>{row['Location']}</b><br/>{row['Time']}<br/>{row['bus']}"

        folium.Circle(
            location=coords,
            fill_opacity=0.6,
            radius=75,
            fill_color=route_color,
            color=route_color,
            fill=0.6,
            tooltip=folium.Tooltip(tooltip_content) 
        ).add_to(map_object)

# create Folium maps
def create_map(group):
    filtered_df = df[df['group'] == group]

    if not filtered_df.empty:
        initial_coords = [44.25382,	-72.7870551]

        m = folium.Map(location=initial_coords, tiles='cartodbpositron', zoom_start=11)
        plot_bus_route(filtered_df, m)

        map_filename = f'{group}_map.html'
        m.save(map_filename)
        
        return map_filename
    else:
        return None

for route_label in df['group'].unique():
    create_map(route_label)